In [112]:
import numpy as np
import pandas as pd


from NPIR import NPIR

from sklearn.utils import shuffle
from sklearn import datasets
from sklearn.datasets import make_blobs
from sklearn import metrics

import matplotlib.pyplot as plt

import datetime
import warnings
from collections import Counter as Cs
from timeit import default_timer as timer
from datetime import timedelta


import findspark
findspark.init()

########
from numpy import array
from math import sqrt
from pyspark.mllib.clustering import KMeans, KMeansModel
########

from pyspark import SparkContext
from operator import *
from pyspark import StorageLevel
from pyspark.sql.types import IntegerType, FloatType, BooleanType, StringType, StructType,\
StructField,ArrayType, DataType
from pyspark.sql.functions import udf, log, rand, monotonically_increasing_id, col, broadcast,\
greatest, desc, asc, row_number, avg, mean, least, struct, lit, sequence, sum
from functools import reduce
import pyspark.sql.functions as F
from pyspark.sql import SparkSession, SQLContext, Window, Row, DataFrame
from pyspark import SparkConf

In [113]:
spark = SparkSession.builder.master("local[*]").config("spark.sql.broadcastTimeout", "30000s").\
config("spark.network.timeout","30000s").config("spark.executor.heartbeatInterval","12000000ms").\
config("spark.storage.blockManagerSlaveTimeoutMs","12000001ms").config("spark.driver.maxResultSize","12g").\
config("spark.default.parallelism", "100").config("spark.memory.offHeap.enabled","true").\
config("spark.memory.offHeap.size", "14g").appName("NPIR_Parallel").getOrCreate()

In [114]:
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [115]:
x1,y1 = make_blobs(n_samples=100, centers=3, n_features=2,
                random_state=1)
data = pd.DataFrame(x1)
data['label'] = y1
data.head()

,0,1,label
0,-0.794152,2.104951,0
1,-9.151552,-4.812864,1
2,-11.441826,-4.457814,1
3,-9.767618,-3.191337,1
4,-4.536556,-8.401863,2


In [116]:
data = shuffle(data)
data.head()

,0,1,label
49,-1.782450,3.470720,0
74,0.000242,5.148534,0
76,-6.195996,-7.402816,2
47,0.526016,3.009994,0
58,-1.617346,4.989305,0


In [117]:
data.to_csv('blobs.csv', index=False)

In [118]:
#read csv
data_spark_df = spark.read.format('csv').option('header','True').option('index','False').load('blobs.csv')
data_spark_df.show()

+--------------------+-------------------+-----+
|                   0|                  1|label|
+--------------------+-------------------+-----+
| -1.7824501314671677| 3.4707204345840927|    0|
|0.000242271161351...|  5.148534029420497|    0|
|  -6.195996026651871| -7.402816464759037|    2|
|  0.5260155005846419|  3.009993533355024|    0|
| -1.6173461592329268| 4.9893050825589835|    0|
|  -6.816083022269968| -8.449869256994909|    2|
| -1.9819771099620271|  4.022435514174746|    0|
|  -9.204905637733754|-4.5768792770429965|    1|
| -7.2013269275537715| -8.272282292398854|    2|
| -1.9274479855745354| 4.9368453355813475|    0|
|  -7.374998957175799|-10.588065868731183|    2|
|-0.19745196890354544| 2.3463491593455075|    0|
|  -0.757969185355724|  4.908984207745029|    0|
|  -6.569670859679778| -8.327931264366546|    2|
| -1.6087521511724905|  3.769494222273808|    0|
| -11.227770639320063| -3.402811051386989|    1|
|  -8.798794623751593|-3.7681921298792607|    1|
| -1.539400953466890

In [119]:
data_spark_df.count()

100

In [120]:
data_spark_df.describe()

DataFrame[summary: string, 0: string, 1: string, label: string]

In [121]:
data_spark_df.printSchema()

root
 |-- 0: string (nullable = true)
 |-- 1: string (nullable = true)
 |-- label: string (nullable = true)



In [122]:
data_spark_df = data_spark_df.select('0', '1')

In [123]:
new_name = ['first', 'second']
data_spark_rdd = data_spark_df.toDF(*new_name).rdd.filter(lambda x:x)
data_spark_df = data_spark_df.toDF(*new_name)
data_spark_df.show()

+--------------------+-------------------+
|               first|             second|
+--------------------+-------------------+
| -1.7824501314671677| 3.4707204345840927|
|0.000242271161351...|  5.148534029420497|
|  -6.195996026651871| -7.402816464759037|
|  0.5260155005846419|  3.009993533355024|
| -1.6173461592329268| 4.9893050825589835|
|  -6.816083022269968| -8.449869256994909|
| -1.9819771099620271|  4.022435514174746|
|  -9.204905637733754|-4.5768792770429965|
| -7.2013269275537715| -8.272282292398854|
| -1.9274479855745354| 4.9368453355813475|
|  -7.374998957175799|-10.588065868731183|
|-0.19745196890354544| 2.3463491593455075|
|  -0.757969185355724|  4.908984207745029|
|  -6.569670859679778| -8.327931264366546|
| -1.6087521511724905|  3.769494222273808|
| -11.227770639320063| -3.402811051386989|
|  -8.798794623751593|-3.7681921298792607|
| -1.5394009534668904|  5.023692978550581|
|  -4.874182454688006|-10.049589027515138|
|  -5.733425071070147| -8.440535968100065|
+----------

In [124]:
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)
spark.conf.set('spark.jars.packages','com.databricks:spark-cav_2.11')
spark.conf.set("spark.sql.parquet.compression.codec","gzip")
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "False")
sqlContext.setConf("spark.sql.shuffle.partitions", "100")

In [125]:
points = data_spark_df.count()
points

100

In [126]:
#IR: The indexing ratio to be used for generating the maximum index
IR = 0.2
#The number of iteration i
i = 10
k = 3 #k: Number of clusters
# count = Cs()
chunk = 50

In [127]:
from pyspark.sql.functions import desc, row_number, monotonically_increasing_id
from pyspark.sql.window import Window

data_spark = data_spark_df.withColumn('index_column_name', row_number().\
                                          over(Window.orderBy(monotonically_increasing_id())) - 1)
data_spark.show()

+--------------------+-------------------+-----------------+
|               first|             second|index_column_name|
+--------------------+-------------------+-----------------+
| -1.7824501314671677| 3.4707204345840927|                0|
|0.000242271161351...|  5.148534029420497|                1|
|  -6.195996026651871| -7.402816464759037|                2|
|  0.5260155005846419|  3.009993533355024|                3|
| -1.6173461592329268| 4.9893050825589835|                4|
|  -6.816083022269968| -8.449869256994909|                5|
| -1.9819771099620271|  4.022435514174746|                6|
|  -9.204905637733754|-4.5768792770429965|                7|
| -7.2013269275537715| -8.272282292398854|                8|
| -1.9274479855745354| 4.9368453355813475|                9|
|  -7.374998957175799|-10.588065868731183|               10|
|-0.19745196890354544| 2.3463491593455075|               11|
|  -0.757969185355724|  4.908984207745029|               12|
|  -6.569670859679778| -

In [128]:
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [129]:
leaderheadr = ['chunkLabel', 'old label', 'count']
leaderheadr.extend([str(x) for x in range(1, len(data_spark.columns))])
leaderheadr = tuple(leaderheadr)
leaderheadr

('chunkLabel', 'old label', 'count', '1', '2')

In [130]:
labelsheader = ('chunkLabel', 'label')
labelsheader

('chunkLabel', 'label')

In [131]:
start = timer()


labels = sqlContext.createDataFrame([np.full(len(labelsheader), np.nan).tolist()],labelsheader)
labels = labels.na.drop()

leaders = sqlContext.createDataFrame([np.full(len(leaderheadr), np.nan).tolist()],leaderheadr)
leaders = leaders.na.drop()

ii = 0
for z in range(0, points, chunk):
    j = z + chunk
    data = data_spark.where(col("index_column_name").between(z, j-1)).toPandas()
    data.drop("index_column_name",axis=1,inplace=True)
    data = data.astype(float)
    from NPIR import NPIR
    label = NPIR(data.values,k,IR,i)
    data['labels'] = label
    
    # Adding to pyspard label
    chunklabel = np.full(len(label), ii).tolist()
    labelDF = [(x, y) for x, y in zip(chunklabel, label)]
    labelsDF = sqlContext.createDataFrame(labelDF,labelsheader)
    labels = unionAll(labels, labelsDF)

    leader = []
    f = list(Cs(label))
    f.sort()
    for i in f:
        leader.append([round(np.mean(z), 4) for z in data[data['labels']==i].values[:,:-1].T])
    del data
    del NPIR
    
    # Adding to pyspark leaders
    for x in range(len(leader)):
        x1 = [ii, x, Cs(label)[x]]
        x1.extend(leader[x])
        leader[x] = x1
    leaderDF = sqlContext.createDataFrame(leader,leaderheadr)
    leaders = unionAll(leaders, leaderDF)
    ii += 1
end = timer()
print ("Execution time HH:MM:SS:",timedelta(seconds=end-start))

Execution time HH:MM:SS: 0:00:00.903166


In [132]:
#leaders.show()

In [133]:
#labels.show()

In [134]:
#labels.count()

In [ ]:
# # Load and parse the data
# data = sc.textFile("data/mllib/kmeans_data.txt")
# parsedData = data.map(lambda line: array([float(x) for x in line.split(' ')]))
parsedData = leaders.select(['1', '2']).rdd.map(list)

start = timer()
# Build the model (cluster the data)
clusters = KMeans.train(parsedData, k, maxIterations=20, initializationMode="random")
end = timer()
print ("Execution time HH:MM:SS:",timedelta(seconds=end-start))

In [ ]:
def Euclidean_distance(a,b,c,d):
    a = float(a)
    b = float(b)
    c = float(c)
    d = float(d)
    array1 = np.array([a,b])
    array2 = np.array([c,d])
    dist = np.linalg.norm(array1-array2)
    dist = dist.item()
    return dist

Euclidean_distance_spark = udf(lambda x,y,z,t: Euclidean_distance(x,y,z,t), FloatType())
sqlContext.sql("SET spark.sql.autoBroadcastJoinThreshold = -1")

In [ ]:
start = timer()

spark_cluster_centroid = sqlContext.createDataFrame(([c.tolist() for c in clusters.centers]),\
                                                    ['cent_x', 'cent_y'])
# spark_cluster_centroid.show()
spark_cluster_centroid = spark_cluster_centroid.withColumn('defined_cluster', row_number().\
                                          over(Window.orderBy(monotonically_increasing_id())) - 1)
# spark_cluster_centroid.show()
data_cent = 0
for i in range(k):
    u = [ str(i)+'x',str(i)+'y']
    centroids = spark_cluster_centroid.filter(col('defined_cluster') == str(i)).\
    drop('defined_cluster').toDF(*u)
#     centroids.show()
    if i == 0:
        data_cent = data_spark_df.join(broadcast(centroids))        
#         data_cent.show()
        data_cent = data_cent.withColumn(str(i),Euclidean_distance_spark(data_cent.columns[0],\
                            data_cent.columns[1],data_cent.columns[i+2],data_cent.columns[i+3]))
#         data_cent.show()
        data_cent = data_cent.drop(data_cent.columns[i+2]).drop(data_cent.columns[i+3])
#         data_cent.show()
        data_cent = data_cent.withColumn('mindist',col(str(i)))
#         data_cent.show()
        data_cent = data_cent.withColumn('mindist1',least(data_cent.columns[i+2], col('mindist')))
#         data_cent.show()
        data_cent = data_cent.drop('mindist')
        data_cent = data_cent.withColumnRenamed('mindist1','mindist')
#         data_cent
    elif i > 0:
        data_cent = data_cent.join(broadcast(centroids))
#         data_cent.show()
        data_cent = data_cent.withColumn(str(i),Euclidean_distance_spark(data_cent.columns[0],\
                                    data_cent.columns[1], data_cent.columns[i+3],data_cent.columns[i+4]))
#         data_cent.show()
        data_cent = data_cent.drop(u[0]).drop(u[1])
#         data_cent.show()
        data_cent = data_cent.withColumn('mindist1',least(data_cent.columns[i+3], col('mindist')))#4
#         data_cent.show()
        data_cent = data_cent.drop('mindist')
        data_cent = data_cent.withColumnRenamed('mindist1','mindist')
        
data_cent = data_cent.drop('mindist')

# data_cent.show()

In [ ]:
def Min_COl( *row_list):
    ind = row_list.index(min(*row_list))
    return int(ind)
find_min_val_name = udf(Min_COl, IntegerType())

data_cent = data_cent.withColumn('defined_cluster', find_min_val_name(*data_cent.columns[2:3+k]))
#data_cent.show()
data_cent = data_cent.select('first','second','defined_cluster')

In [ ]:
data_cent

In [ ]:
#data_cent.show()

In [ ]:
d = data_cent.toPandas()

end = timer()
print ("Execution time HH:MM:SS:",timedelta(seconds=end-start))

In [ ]:
data = pd.read_csv('blobs.csv')
data.head()

In [ ]:
data = data[['0','1','label']]

In [ ]:
data['new'] = d['defined_cluster'].astype('int8')
data.head()

In [ ]:
y = data['label']
labelsPred = data['new']

In [ ]:
x = data[['0','1']].values
x.shape

In [ ]:
#printing results
#print('labels:')
# print(labelsPred)

# tEnd = datetime.datetime.now()
# print('Time: ' + str(tEnd - tStart))
print('Measures:')
print('HS: ' + str(metrics.homogeneity_score(y,labelsPred)))
print('CS: ' + str(metrics.completeness_score(y,labelsPred)))
print('VM: ' + str(metrics.v_measure_score(y,labelsPred)))
print('AMI: ' + str(metrics.adjusted_mutual_info_score(y,labelsPred)))
print('ARI: ' + str(metrics.adjusted_rand_score(y,labelsPred)))

In [ ]:
import matplotlib.pyplot as plt
from itertools import cycle, islice 

fig = plt.figure()      
colors = np.array(list(islice(cycle(['#377eb8', '#ff7f00', '#4daf4a','#f781bf', '#a65628', '#984ea3',
                                    '#999999', '#e41a1c', '#dede00']),int(k))))
plt.scatter(x[:, 0], x[:, 1], s=10, color=colors[labelsPred.tolist()])
plt.show()
# plt.clf()